In [6]:
import pandas as pd
from psycopg2 import sql
from psycopg2.extras import execute_values
import config

In [11]:
db_config = config.load_config()
conn = config.connect(db_config)
conn.autocommit = True
cursor = conn.cursor()

Connected to the PostgreSQL server.


In [5]:
cursor.execute("CREATE DATABASE health")
cursor.close()
conn.close()

In [ ]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS activity (
        _id SERIAL PRIMARY KEY,
        date DATE NOT NULL,
        time TIME NOT NULL,
        activity INT,
        acceleration_x FLOAT NOT NULL,
        acceleration_y FLOAT NOT NULL,
        acceleration_z FLOAT NOT NULL,
        gyro_x FLOAT NOT NULL,
        gyro_y FLOAT NOT NULL,
        gyro_z FLOAT NOT NULL
    );
    """)

In [13]:
file_path = '../training/training.data'
training_df = pd.read_csv(file_path, delimiter=';', header=0)

# Conversion of columns : 

# column date : 
training_df['date'] = pd.to_datetime(training_df['date'], format='%d/%m/%y').dt.strftime('%Y-%m-%d')

# column activity : 
training_df['activity'] = training_df['activity'].astype(int)

# Column time :
converted_time = []
for time_value in training_df['time']:
    try:
        parts = time_value.split(':')
        if len(parts) == 4:
            time_str = f"{parts[0]}:{parts[1]}:{parts[2]}.{parts[3]}"
            converted_time.append(time_str)
        else:
            raise ValueError(f"Format expected : {time_value}")
    except Exception as e:
        print(f"Error while loading the time {time_value}: {e}")
        converted_time.append(None)

training_df['time'] = converted_time


# Explicit type convertion of column in native python
training_df = training_df.astype({
    'activity': int,
    'acceleration_x': float,
    'acceleration_y': float,
    'acceleration_z': float,
    'gyro_x': float,
    'gyro_y': float,
    'gyro_z': float,
})

In [14]:
# Conversion of lines of DataFrame in tuple list (with activity in boolean)
records = [tuple(map(lambda x: x.item() if hasattr(x, 'item') else x, row)) for row in training_df.to_records(index=False)]

records = [
    (
        row.date,
        row.time,
        int(row.activity),
        row.acceleration_x,
        row.acceleration_y,
        row.acceleration_z,
        row.gyro_x,
        row.gyro_y,
        row.gyro_z,
    )
    for row in training_df.itertuples(index=False)
]


# Insertion request
query = """
    INSERT INTO activity (date, time, activity, acceleration_x, acceleration_y, acceleration_z, gyro_x, gyro_y, gyro_z)
    VALUES %s
"""

# Execution of the insertion request
execute_values(cursor, query, records)

In [ ]:
cursor.close()
conn.close()